In [46]:
import pandas as pd
import numpy as np
from datetime import datetime
import missingno as msno

In [1]:
import load_data as data

loans = data.loans
print(loans.shape)

(54231, 43)


In [107]:
class DataTransform:
    def __init__(self, series):
        self.series = series 

    def return_series(self):
        return self.series

    def display_series(self):
        print(self.series.head(3))
        print("Non-null count:", self.series.count())
        return self

    def to_date(self, date_format):
        self.date_format = date_format
        self.series = pd.to_datetime(self.series, format=date_format)
        # Format the result as ISO dates with month precision
        self.series = self.series.dt.strftime("%Y-%m")
        return self
    
    def to_timedelta(self):
        int_series = self.series.apply(lambda x: int(x) if not pd.isna(x) else pd.NaT)
        days_in_month = 30
        timedelta_arr = np.where(int_series.notna(), pd.to_timedelta(int_series * days_in_month, unit='D'), pd.NaT)
        timedelta_ser = pd.Series(timedelta_arr)
        self.series = timedelta_ser.astype('timedelta64[s]')
        return self

    def to_category(self):
        self.series = self.series.astype('category')
        return self

    def to_num(self):
        self.series = pd.to_numeric(self.series, errors='coerce')
        return self
    
    def round(self):
        self.series = self.series.round(0)
        return self
    
    def to_int(self):
        self.series = self.series.astype('int64')
        return self
    # This will only work if there are no missing values, because the type of NaN is float.

    def split_value(self):
        self.series = self.series.apply(lambda x: x.split()[0] if isinstance(x, str) else x)
        return self


In [115]:
# Convert 'mths_since_last_major_derog' from float to timedelta.
print("Initial data non-null count:", loans["mths_since_last_major_derog"].count())
mslmd_transform = DataTransform(loans["mths_since_last_major_derog"])
mslmd_timedelta = mslmd_transform.to_timedelta().display_series().return_series()
loans["mths_since_last_major_derog"] = mslmd_timedelta

Initial data non-null count: 7499
0   150 days 00:00:00
1                 NaT
2                 NaT
dtype: timedelta64[s]
Non-null count: 7499


In [112]:
# Convert 'last_credit_pull_date' from object to datetime64, format as date string YYYY-MM.
print("Initial data non-null count:", loans["last_credit_pull_date"].count())
last_credit_pull_date_transform = DataTransform(loans["last_credit_pull_date"])
last_credit_pull_iso = last_credit_pull_date_transform.to_date("%b-%Y").display_series().return_series()
loans["last_credit_pull_date"] = last_credit_pull_iso

Initial data non-null count: 54224
0    2022-01
1    2022-01
2    2021-10
Name: last_credit_pull_date, dtype: object
Non-null count: 54224


In [109]:
# Convert 'next_payment_date' from object to datetime64, format as date string YYYY-MM.
print("Initial data non-null count:", loans["next_payment_date"].count())
next_payment_date_transform = DataTransform(loans["next_payment_date"])
next_payment_iso = next_payment_date_transform.to_date("%b-%Y").display_series().return_series()
loans["next_payment_date"] = next_payment_iso

Initial data non-null count: 21623
0    2022-02
1    2022-02
2        NaN
Name: next_payment_date, dtype: object
Non-null count: 21623


In [105]:
# Convert 'last_payment_date' from object to datetime64, format as date string YYYY-MM.
last_payment_date_transform = DataTransform(loans["last_payment_date"])
last_payment_iso = last_payment_date_transform.to_date("%b-%Y").display_series().return_series()
loans["last_payment_date"] = last_payment_iso

0    2022-01
1    2022-01
2    2021-10
Name: last_payment_date, dtype: object
54158


In [101]:
# Convert 'mths_since_last_record' from float to timedelta.
mslr_transform = DataTransform(loans["mths_since_last_record"])
mslr_timedelta = mslr_transform.to_timedelta().display_series().return_series()
loans["mths_since_last_record"] = mslr_timedelta

0   NaT
1   NaT
2   NaT
dtype: timedelta64[s]
6181


In [96]:
# Convert 'mths_since_last_delinq' from float to timedelta.
msld_transform = DataTransform(loans["mths_since_last_delinq"])
msld_timedelta = msld_transform.to_timedelta().display_series().return_series()
loans["mths_since_last_delinq"] = msld_timedelta

0    150 days 00:00:00
1                  NaT
2   2070 days 00:00:00
dtype: timedelta64[s]
23229


In [81]:
# Convert 'earliest_credit_line' from object to datetime64, format as date string YYYY-MM
earliest_credit_line_transform = DataTransform(loans["earliest_credit_line"])
earliest_credit_line_iso = earliest_credit_line_transform.to_date("%b-%Y").display_series().return_series()
loans["earliest_credit_line"] = earliest_credit_line_iso

0    1987-10
1    2001-09
2    1998-09
Name: earliest_credit_line, dtype: object
54231


In [78]:
# Convert 'payment_plan' from object to category datatype.
payment_plan_transform = DataTransform(loans["payment_plan"])
payment_plan_cat = payment_plan_transform.to_category().display_series().return_series()
loans["payment_plan"] = payment_plan_cat

0    n
1    n
2    n
Name: payment_plan, dtype: category
Categories (2, object): ['n', 'y']
54231


In [74]:
# Convert 'issue_date' from object to datetime64, format as date string YYYY-MM
issue_date_transform = DataTransform(loans["issue_date"])
issue_date_iso = issue_date_transform.to_date("%b-%Y").display_series().return_series()
print(issue_date_iso.dtypes)

0    2021-01
1    2021-01
2    2021-01
Name: issue_date, dtype: object
54231
object


In [75]:
loans["issue_date"] = issue_date_iso

In [53]:
# Convert 'home_ownership' from object to category
home_ownership_transform = DataTransform(loans["home_ownership"])
home_ownership_cat = home_ownership_transform.to_category().display_series().return_series()

0    MORTGAGE
1        RENT
2    MORTGAGE
Name: home_ownership, dtype: category
Categories (5, object): ['MORTGAGE', 'NONE', 'OTHER', 'OWN', 'RENT']
54231


In [54]:
loans["home_ownership"] = home_ownership_cat

In [49]:
# Convert 'employment_length' from object to category datatype. 
# Would prefer to drop years and make ordinal, but values are awkward.
employment_length_transform = DataTransform(loans["employment_length"])
employment_length_years = employment_length_transform.to_category().display_series().return_series()

0    5 years
1    9 years
2    8 years
Name: employment_length, dtype: category
Categories (11, object): ['1 year', '10+ years', '2 years', '3 years', ..., '7 years', '8 years', '9 years', '< 1 year']
52113


In [51]:
loans["employment_length"] = employment_length_years

In [40]:
# Convert sub_grade from object to category datatype.
subgrade_transform = DataTransform(loans["sub_grade"])
subgrade_cat = subgrade_transform.to_category().display_series().return_series()

0    A4
1    A3
2    A4
Name: sub_grade, dtype: category
Categories (35, object): ['A1', 'A2', 'A3', 'A4', ..., 'G2', 'G3', 'G4', 'G5']
54231


In [41]:
loans["sub_grade"] = subgrade_cat

In [ ]:
# From series "term": remove 'months' part of string and convert to numeric datatype.
transform_term = DataTransform(loans["term"])
# Create as new column
loans["term_months"] = transform_term.split_value().to_num().round().display_series().return_series()

In [ ]:
# Replace original column
loans["term"] = loans["term_months"]

In [ ]:
# Remove redundant column
loans = loans.drop(columns=["term_months"])

In [23]:
# Convert series 'grade' to category datatype. 
transform_grade = DataTransform(loans["grade"])
loans["grade_cat"] = transform_grade.to_category().display_series().return_series()

0    A
1    A
2    A
3    C
4    A
Name: grade, dtype: category
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [25]:
loans["grade"] = loans["grade_cat"]
loans = loans.drop(columns=["grade_cat"])

In [116]:
# Check data here
loans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           54231 non-null  int64         
 1   member_id                    54231 non-null  int64         
 2   loan_amount                  54231 non-null  int64         
 3   funded_amount                51224 non-null  float64       
 4   funded_amount_inv            54231 non-null  float64       
 5   term                         49459 non-null  float64       
 6   int_rate                     49062 non-null  float64       
 7   instalment                   54231 non-null  float64       
 8   grade                        54231 non-null  category      
 9   sub_grade                    54231 non-null  category      
 10  employment_length            52113 non-null  category      
 11  home_ownership               54231 non-null  c